In [1]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from calendar import monthrange
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

In [35]:
df = pd.read_csv("https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_02.csv")

In [36]:
df_sample = df[df["region"]=="CENTRE VAL DE LOIRE"]

In [37]:
df_stations_old = df_sample.groupby(["NUM_POSTE"]).max()
print(len(df_stations_old))
display(df_stations_old)

39


,NOM_USUEL,LAT,LON,ALTI,AAAAMM,Year,Month,precip_cumul_mensu,temp_mean_mensu,temp_max_mensu,...,vent_speed_inst_moy_mensu,vent_speed_inst_max_mensu,vent_dir_inst,vent_nbjour_inst_speedsup10ms,vent_speed_10mn_max_mensu,vent_dir_10mn,vent_nbjour_10mn_speedsup10ms,departement_num,departement_name,region
NUM_POSTE,,,,,,,,,,,,,,,,,,,,,
4134002,LA MOTTE DU CAIRE,44.326667,6.030500,665,2022-12-01,2022,12,277.0,33.7,40.0,...,6.8,103.3,360.0,20.0,46.8,360.0,2.0,41,Loir-et-Cher,CENTRE VAL DE LOIRE
4136001,LA MURE-ARGENS,43.977000,6.520167,920,2022-12-01,2022,12,332.7,31.5,37.3,...,8.3,117.7,360.0,29.0,54.4,350.0,4.0,41,Loir-et-Cher,CENTRE VAL DE LOIRE
4157001,PUIMOISSON,43.871167,6.168833,775,1993-05-01,1993,12,226.8,29.6,34.0,...,17.3,108.0,360.0,30.0,61.2,360.0,9.0,41,Loir-et-Cher,CENTRE VAL DE LOIRE
4173002,ST ANDRE ALPES,43.958333,6.511667,882,2005-06-01,2005,12,320.0,30.9,35.8,...,10.8,115.2,360.0,30.0,50.4,360.0,8.0,41,Loir-et-Cher,CENTRE VAL DE LOIRE
4192001,ST MICHEL L OBS,43.933333,5.713333,650,2003-10-01,2003,12,242.6,32.1,37.0,...,14.4,93.6,360.0,28.0,46.8,360.0,5.0,41,Loir-et-Cher,CENTRE VAL DE LOIRE
18015003,AUBIGNY-SUR-NERE,47.499500,2.427333,176,2022-12-01,2022,12,257.2,31.3,41.1,...,18.0,136.4,340.0,27.0,69.1,340.0,11.0,18,Cher,CENTRE VAL DE LOIRE
18033001,BOURGES,47.059167,2.359833,161,2022-12-01,2022,12,186.6,31.9,41.7,...,23.0,201.6,360.0,27.0,86.4,360.0,15.0,18,Cher,CENTRE VAL DE LOIRE
18092001,AVORD,47.052167,2.642167,175,2022-12-01,2022,12,225.1,31.7,40.9,...,23.0,169.2,360.0,28.0,100.8,360.0,19.0,18,Cher,CENTRE VAL DE LOIRE
18125004,LERE,47.457833,2.893167,139,2022-12-01,2022,12,162.0,31.9,41.9,...,14.0,110.5,360.0,26.0,63.7,360.0,9.0,18,Cher,CENTRE VAL DE LOIRE


In [38]:
# filter out station qui n'ont pas de données récentes (2022)
last_year = df_sample["Year"].max()
mask = df_stations_old["Year"]!=last_year
df_stations_old = df_stations_old[mask]
stations_old = df_stations_old.index.to_list()
df_recent = df_sample[~df_sample["NUM_POSTE"].isin(stations_old)]

In [39]:
print(len(df_recent.groupby(["NUM_POSTE"]).max()))
df_recent.groupby(["NUM_POSTE"]).max()

30


,NOM_USUEL,LAT,LON,ALTI,AAAAMM,Year,Month,precip_cumul_mensu,temp_mean_mensu,temp_max_mensu,...,vent_speed_inst_moy_mensu,vent_speed_inst_max_mensu,vent_dir_inst,vent_nbjour_inst_speedsup10ms,vent_speed_10mn_max_mensu,vent_dir_10mn,vent_nbjour_10mn_speedsup10ms,departement_num,departement_name,region
NUM_POSTE,,,,,,,,,,,,,,,,,,,,,
4134002,LA MOTTE DU CAIRE,44.326667,6.030500,665,2022-12-01,2022,12,277.0,33.7,40.0,...,6.8,103.3,360.0,20.0,46.8,360.0,2.0,41,Loir-et-Cher,CENTRE VAL DE LOIRE
4136001,LA MURE-ARGENS,43.977000,6.520167,920,2022-12-01,2022,12,332.7,31.5,37.3,...,8.3,117.7,360.0,29.0,54.4,350.0,4.0,41,Loir-et-Cher,CENTRE VAL DE LOIRE
18015003,AUBIGNY-SUR-NERE,47.499500,2.427333,176,2022-12-01,2022,12,257.2,31.3,41.1,...,18.0,136.4,340.0,27.0,69.1,340.0,11.0,18,Cher,CENTRE VAL DE LOIRE
18033001,BOURGES,47.059167,2.359833,161,2022-12-01,2022,12,186.6,31.9,41.7,...,23.0,201.6,360.0,27.0,86.4,360.0,15.0,18,Cher,CENTRE VAL DE LOIRE
18092001,AVORD,47.052167,2.642167,175,2022-12-01,2022,12,225.1,31.7,40.9,...,23.0,169.2,360.0,28.0,100.8,360.0,19.0,18,Cher,CENTRE VAL DE LOIRE
18125004,LERE,47.457833,2.893167,139,2022-12-01,2022,12,162.0,31.9,41.9,...,14.0,110.5,360.0,26.0,63.7,360.0,9.0,18,Cher,CENTRE VAL DE LOIRE
18172003,ORVAL RAD,46.731000,2.467333,165,2022-12-01,2022,12,164.3,31.9,42.1,...,13.7,123.8,360.0,25.0,63.7,360.0,5.0,18,Cher,CENTRE VAL DE LOIRE
18175003,OUROUER,46.925333,2.803333,221,2022-12-01,2022,12,195.0,31.4,40.6,...,18.7,127.1,360.0,26.0,79.2,330.0,11.0,18,Cher,CENTRE VAL DE LOIRE
18187004,PREVERANGES,46.425333,2.236500,462,2022-12-01,2022,12,203.1,29.0,39.0,...,18.7,127.8,360.0,26.0,68.4,320.0,11.0,18,Cher,CENTRE VAL DE LOIRE


In [40]:
# check continuité des données sur 7 ans
stations = df_recent["NUM_POSTE"].unique()
to_drop = []
for num_station in stations:
    df_station = df_recent[df_recent["NUM_POSTE"]==num_station]

    for year in range(last_year,last_year-7,-1):
        nb_missing_month = 0
        try:
            df_station_year = df_station[df_station["Year"]==year]
            nb_missing_month += 12-df_station_year["Month"].count()
            if nb_missing_month > 0:
                to_drop.append(num_station)
                print(f"Drop station N°{num_station}")
                print(f"{nb_missing_month} manquants en {year}\n\n")
            else:
                print(f"Station N°{num_station} has {df_station_year["Month"].count()} months in {year}")

        except Exception as e:
            to_drop.append(num_station)
            print(e)
            print()
            print(f"Drop station N°{num_station}")
            print(f"{year} est manquante\n\n")

    print()

Station N°4134002 has 12 months in 2022
Station N°4134002 has 12 months in 2021
Station N°4134002 has 12 months in 2020
Station N°4134002 has 12 months in 2019
Station N°4134002 has 12 months in 2018
Station N°4134002 has 12 months in 2017
Station N°4134002 has 12 months in 2016

Station N°4136001 has 12 months in 2022
Station N°4136001 has 12 months in 2021
Station N°4136001 has 12 months in 2020
Station N°4136001 has 12 months in 2019
Station N°4136001 has 12 months in 2018
Station N°4136001 has 12 months in 2017
Station N°4136001 has 12 months in 2016

Station N°18015003 has 12 months in 2022
Station N°18015003 has 12 months in 2021
Station N°18015003 has 12 months in 2020
Station N°18015003 has 12 months in 2019
Station N°18015003 has 12 months in 2018
Station N°18015003 has 12 months in 2017
Station N°18015003 has 12 months in 2016

Station N°18033001 has 12 months in 2022
Station N°18033001 has 12 months in 2021
Station N°18033001 has 12 months in 2020
Station N°18033001 has 12 m

In [41]:
to_drop

[41173003]

In [42]:
df_recent = df_recent[~df_recent["NUM_POSTE"].isin(to_drop)]

In [25]:
len(df_recent["NUM_POSTE"].unique())

29

In [43]:
# PREPROCESSING DATE

df_recent["AAAAMM"] = pd.to_datetime(df_recent["AAAAMM"])

def last_day_of_month(aaaamm):
    """
    prend une date et retourne une string avec le jour en dernier jour du mois
    """
    year = aaaamm.year
    month = aaaamm.month
    last_day = monthrange(year, month)[1]
    return f"{year}-{month:02d}-{last_day:02d}"

df_recent["date"] = df_recent["AAAAMM"].apply(last_day_of_month)
df_recent["date"] = pd.to_datetime(df_recent["date"])


In [44]:
to_keep = ["NUM_POSTE","NOM_USUEL","LAT","LON","ALTI","Year","Month","vent_speed_inst_moy_mensu","departement_num","departement_name","region","date"]
to_drop = [col for col in df_sample if col not in to_keep]

In [45]:
df_recent = df_recent.drop(to_drop,axis=1)

In [47]:
df_recent

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,date
11437,4134002,LA MOTTE DU CAIRE,44.326667,6.030500,665,1990,1,3.6,41,Loir-et-Cher,CENTRE VAL DE LOIRE,1990-01-31
11438,4134002,LA MOTTE DU CAIRE,44.326667,6.030500,665,1990,2,4.3,41,Loir-et-Cher,CENTRE VAL DE LOIRE,1990-02-28
11439,4134002,LA MOTTE DU CAIRE,44.326667,6.030500,665,1990,3,5.4,41,Loir-et-Cher,CENTRE VAL DE LOIRE,1990-03-31
11440,4134002,LA MOTTE DU CAIRE,44.326667,6.030500,665,1990,4,5.8,41,Loir-et-Cher,CENTRE VAL DE LOIRE,1990-04-30
11441,4134002,LA MOTTE DU CAIRE,44.326667,6.030500,665,1990,5,5.0,41,Loir-et-Cher,CENTRE VAL DE LOIRE,1990-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...
181726,45340002,VILLEMURLIN,47.693500,2.366167,144,2022,8,6.5,45,Loiret,CENTRE VAL DE LOIRE,2022-08-31
181727,45340002,VILLEMURLIN,47.693500,2.366167,144,2022,9,6.1,45,Loiret,CENTRE VAL DE LOIRE,2022-09-30
181728,45340002,VILLEMURLIN,47.693500,2.366167,144,2022,10,6.8,45,Loiret,CENTRE VAL DE LOIRE,2022-10-31
181729,45340002,VILLEMURLIN,47.693500,2.366167,144,2022,11,8.6,45,Loiret,CENTRE VAL DE LOIRE,2022-11-30


In [95]:
def clean_continuity(dataframe,champ_date):

    """
    INPUT = un dataframe, le nom de sa variable date
    --> transforme en série temporelle à frequence mensuelle
    --> parcours la série d'année en année en commençant par la plus récente
    --> si année en cours < 12 mois, drop l'année et toutes les années antérieures
    OUTPUT = série temporelle clean
    """
    
    dataframe.set_index(champ_date, inplace=True)
    dataframe = dataframe.asfreq("ME")



        

    return dataframe
    

In [74]:
def get_wind_forecast(time, data):

    """
    INPUT =  nombre de mois (time) à prédire et le set de données

    --> prédit la force du vent à horizon "time"
    
    OUTPUT =  le df de test, les prédictions, la MAE et la MAPE
    """
    # calcule date de split 
    date_limite = data.index.max() - pd.DateOffset(months=time)

    # split train / test
    train = data[data.index <= date_limite]
    test = data[data.index > date_limite]

    # split variables exogènes / target
    to_keep = ["LON","LAT","ALTI"]

    var_exog_train = train[to_keep]
    y_train = train["vent_speed_inst_moy_mensu"]

    var_exog_test = test[to_keep]
    y_test = test["vent_speed_inst_moy_mensu"]

    # Vérifier la présence de NaN
    print(data["NUM_POSTE"].unique())
    print(var_exog_train.isna().sum())

    # Vérifier la présence de valeurs infinies
    print(np.isinf(var_exog_train).sum())


    # training
    model = SARIMAX(y_train, exog=var_exog_train, order=(1, 1, 1), seasonal_order=(2, 1, 2, 12))
    sarima_model = model.fit(maxiter=1000)

    # prédictions
    forecast = sarima_model.get_forecast(steps = time, exog = var_exog_test)
    predicted_values = forecast.predicted_mean

    # évaluation
    mae = mean_absolute_error(y_test, predicted_values)
    mape = mean_absolute_percentage_error(y_test, predicted_values)

    # rendu
    return test, predicted_values, mae, mape


In [101]:
df_station = df_recent[df_recent["NUM_POSTE"]==4134002]
df_station_clean = clean_continuity(df_station,"date")

years = df_station["Year"].unique()
years_clean = df_station_clean["Year"].unique()

print(years_clean)
for index,year in enumerate(years_clean):
    print(year)
    print(years_clean[index+1])

[1990. 1991. 1992.   nan 1993. 1994. 1995. 1996. 1997. 1998. 2008. 2009.
 2010. 2011. 2012. 2013. 2014. 2015. 2016. 2017. 2018. 2019. 2020. 2021.
 2022.]
1990.0
1991.0
1991.0
1992.0
1992.0
nan
nan
1993.0
1993.0
1994.0
1994.0
1995.0
1995.0
1996.0
1996.0
1997.0
1997.0
1998.0
1998.0
2008.0
2008.0
2009.0
2009.0
2010.0
2010.0
2011.0
2011.0
2012.0
2012.0
2013.0
2013.0
2014.0
2014.0
2015.0
2015.0
2016.0
2016.0
2017.0
2017.0
2018.0
2018.0
2019.0
2019.0
2020.0
2020.0
2021.0
2021.0
2022.0
2022.0


IndexError: index 25 is out of bounds for axis 0 with size 25

In [75]:
# créé un dictionnaire contenant un dataframe par numéro de station
# chaque dataset est réindexé sur la date avec fréquence de saisonnalité mensuelle

stations = df_recent["NUM_POSTE"].unique()
horizon = 84

dict_pred_eval = {}

for station in stations:
    
    df_station = df_recent[df_recent["NUM_POSTE"]==station]
    df_station = clean_continuity(df_station,"date")

    df_station_eval, predictions, mae, mape = get_wind_forecast(horizon,df_station)
    df_station_eval[f"pred_{horizon}_mois"] = predictions
    df_station_eval["mae"] = mae
    df_station_eval["mape"] = mape

display(dict_pred_eval[stations[0]])


    


    


[4134002.      nan]
LON     123
LAT     123
ALTI    123
dtype: int64
LON     0
LAT     0
ALTI    0
dtype: int64


MissingDataError: exog contains inf or nans

In [ ]:
'''for station in stations:


    df_station = df_sample[df_sample["NUM_POSTE"]==station]
    df_station.set_index("date", inplace=True)
    df_station = df_station.asfreq("ME")  # 'ME' pour mensuel
    
    if len(df_station)-1 >= 84:
        print(f"Station: {station}")
        print(f"nb_mois = {len(df_station)-1}")
        print()
        count_84 += 1
        if df_station.isna().any().any() == True:
            count_NA += 1

print(f"total station avec mois > 84: {count_84}")
print(f"total station > 84 avec NA: {count_NA}")'''

In [18]:
stations = df_sample["NUM_POSTE"].unique()

df_station = df_sample[df_sample["NUM_POSTE"]==stations[0]]

df_station.groupby(["Year"]).count()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,date
Year,,,,,,,,,,,
1990,12,12,12,12,12,12,12,12,12,12,12
1991,12,12,12,12,12,12,12,12,12,12,12
1992,11,11,11,11,11,11,11,11,11,11,11
1993,11,11,11,11,11,11,11,11,11,11,11
1994,12,12,12,12,12,12,12,12,12,12,12
1995,12,12,12,12,12,12,12,12,12,12,12
1996,12,12,12,12,12,12,12,12,12,12,12
1997,12,12,12,12,12,12,12,12,12,12,12
1998,6,6,6,6,6,6,6,6,6,6,6


In [23]:
df_year = df_station[df_station["Year"]==2022]
df_year = df_year["Year"]
df_year.count()


12